In [48]:
import cv2
import mediapipe as mp
import numpy as np
import torch
import joblib
from pathlib import Path

In [72]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="SymbolDatabase.GetPrototype")

In [28]:
# device 선택 (GPU가 있으면 GPU, 없으면 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [30]:
import torch.nn as nn

class FallLSTM(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=2, dropout=0.3, num_classes=2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                            batch_first=True, dropout=dropout, bidirectional=False)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        # x: (B, T, D)
        out, _ = self.lstm(x)       # out: (B, T, hidden)
        last = out[:, -1, :]        # 마지막 타임스텝을 사용 (B, hidden)
        logits = self.fc(last)      # (B, num_classes)
        return logits

In [32]:
BASE_DIR = Path.cwd().parent
ckpt_dir = BASE_DIR / "models"
best_ckpt = ckpt_dir / "best_fall_model.pth"

In [34]:
ck = torch.load(str(best_ckpt), map_location=device)
model = FallLSTM(input_size=132, hidden_size=128, num_layers=2).to(device)
model.load_state_dict(ck["model_state"])
model.eval()

FallLSTM(
  (lstm): LSTM(132, 128, num_layers=2, batch_first=True, dropout=0.3)
  (fc): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=64, out_features=2, bias=True)
  )
)

In [36]:
scaler_path = BASE_DIR / "src" / "scaler.save"  # 앞서 저장한 scaler
scaler = joblib.load(scaler_path) if scaler_path.exists() else None

In [38]:
WINDOW_SIZE = 80

In [50]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    enable_segmentation=False,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)
draw = mp.solutions.drawing_utils

In [52]:
def extract_keypoints(frame):
    """MediaPipe로 33관절 keypoints 추출, flatten 후 반환"""
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)
    if results.pose_landmarks:
        kp = [[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark]
    else:
        kp = [[0,0,0,0]] * 33
    flat = np.array([v for joint in kp for v in joint], dtype=np.float32)
    if scaler is not None:
        flat = scaler.transform(flat.reshape(1,-1)).flatten()
    return flat

In [70]:
def predict_fall_video(video_path):
    cap = cv2.VideoCapture(str(video_path))
    seq_buffer = []  # 시퀀스 버퍼
    frame_idx = 0
    last_label = None
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # 키포인트 추출
        keypoints = extract_keypoints(frame)
        seq_buffer.append(keypoints)
        
        # 시퀀스 길이 맞추기
        if len(seq_buffer) == WINDOW_SIZE:
            x = np.array(seq_buffer, dtype=np.float32)[np.newaxis, ...]  # (1, T, D)
            x_tensor = torch.from_numpy(x).to(device)
            
            with torch.no_grad():
                logits = model(x_tensor)
                probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
                pred = int(np.argmax(probs))
            
            label = "FALL" if pred==1 else "NORMAL"

            print(f"[Frame {frame_idx}] 상태: {label} (확률: {probs[pred]:.2f})")

            #if label != last_label:  # 변화 있을 때만 출력 ← 추가됨
            #    print(f"[Frame {frame_idx}] 상태: {label} (확률: {probs[pred]:.2f})")
            #    last_label = label
                 
            cv2.putText(frame, f"{label}", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,0,255) if pred==1 else (0,255,0), 3)
            
            # 슬라이딩 시퀀스: 한 프레임씩 이동
            seq_buffer.pop(0)
        
        # 영상 보여주기
        cv2.imshow("Fall Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        frame_idx += 1
        
    cap.release()
    cv2.destroyAllWindows()

In [74]:
# -------------------
# 테스트
# -------------------
VIDEO_PATH = Path(BASE_DIR / "data" / "raw" / "LE2i" / "Home_01" / "Videos" / "video (1).avi")
predict_fall_video(VIDEO_PATH)

[Frame 79] 상태: NORMAL (확률: 1.00)
[Frame 80] 상태: NORMAL (확률: 1.00)
[Frame 81] 상태: NORMAL (확률: 1.00)
[Frame 82] 상태: NORMAL (확률: 1.00)
[Frame 83] 상태: NORMAL (확률: 1.00)
[Frame 84] 상태: NORMAL (확률: 1.00)
[Frame 85] 상태: NORMAL (확률: 1.00)
[Frame 86] 상태: NORMAL (확률: 1.00)
[Frame 87] 상태: NORMAL (확률: 1.00)
[Frame 88] 상태: NORMAL (확률: 1.00)
[Frame 89] 상태: NORMAL (확률: 1.00)
[Frame 90] 상태: NORMAL (확률: 1.00)
[Frame 91] 상태: NORMAL (확률: 1.00)
[Frame 92] 상태: NORMAL (확률: 1.00)
[Frame 93] 상태: NORMAL (확률: 1.00)
[Frame 94] 상태: NORMAL (확률: 1.00)
[Frame 95] 상태: NORMAL (확률: 1.00)
[Frame 96] 상태: NORMAL (확률: 1.00)
[Frame 97] 상태: NORMAL (확률: 1.00)
[Frame 98] 상태: NORMAL (확률: 1.00)
[Frame 99] 상태: NORMAL (확률: 1.00)
[Frame 100] 상태: NORMAL (확률: 1.00)
[Frame 101] 상태: NORMAL (확률: 1.00)
[Frame 102] 상태: NORMAL (확률: 1.00)
[Frame 103] 상태: NORMAL (확률: 1.00)
[Frame 104] 상태: NORMAL (확률: 1.00)
[Frame 105] 상태: NORMAL (확률: 1.00)
[Frame 106] 상태: NORMAL (확률: 1.00)
[Frame 107] 상태: NORMAL (확률: 1.00)
[Frame 108] 상태: NORMAL (확률: 1.00)
[

In [12]:
DATA_PATH = BASE_DIR / "data" / "processed" / "npz" / "fall_dataset.npz"
# === npz 데이터 불러오기 ===
print("🔹 테스트 데이터 로드 중...")
data = np.load(DATA_PATH, allow_pickle=True)
X, Y = data["X"], data["Y"]

for _ in range(100):
    # === 한 샘플만 테스트 (낙상 or 비낙상 랜덤) ===
    idx = np.random.randint(0, len(X))
    sample = X[idx]
    label = Y[idx]
    
    # === 스케일 적용 ===
    # scaler는 (N*T, D)에 대해 fit했으므로, frame별로 변환
    sample_scaled = np.array([scaler.transform(frame.reshape(1, -1))[0] for frame in sample])
    
    # === 텐서 변환 ===
    x_tensor = torch.tensor(sample_scaled, dtype=torch.float32).unsqueeze(0).to(device)
    
    # === 예측 ===
    with torch.no_grad():
        logits = model(x_tensor)
        probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
        pred = int(np.argmax(probs))
    
    print("\n==== 🔍 예측 결과 ====")
    print(f"실제 라벨: {label}  →  예측 라벨: {pred}")
    print(f"클래스 확률: 낙상(1): {probs[1]:.4f}, 비낙상(0): {probs[0]:.4f}")
    print("======================")
    
    if pred == 1:
        print("🚨 낙상으로 감지되었습니다!")
    else:
        print("✅ 정상적인 행동입니다.")

🔹 테스트 데이터 로드 중...

==== 🔍 예측 결과 ====
실제 라벨: 0  →  예측 라벨: 0
클래스 확률: 낙상(1): 0.0016, 비낙상(0): 0.9984
✅ 정상적인 행동입니다.

==== 🔍 예측 결과 ====
실제 라벨: 0  →  예측 라벨: 0
클래스 확률: 낙상(1): 0.0002, 비낙상(0): 0.9998
✅ 정상적인 행동입니다.

==== 🔍 예측 결과 ====
실제 라벨: 1  →  예측 라벨: 1
클래스 확률: 낙상(1): 0.9999, 비낙상(0): 0.0001
🚨 낙상으로 감지되었습니다!

==== 🔍 예측 결과 ====
실제 라벨: 1  →  예측 라벨: 1
클래스 확률: 낙상(1): 0.9999, 비낙상(0): 0.0001
🚨 낙상으로 감지되었습니다!

==== 🔍 예측 결과 ====
실제 라벨: 0  →  예측 라벨: 0
클래스 확률: 낙상(1): 0.0002, 비낙상(0): 0.9998
✅ 정상적인 행동입니다.

==== 🔍 예측 결과 ====
실제 라벨: 1  →  예측 라벨: 1
클래스 확률: 낙상(1): 0.9999, 비낙상(0): 0.0001
🚨 낙상으로 감지되었습니다!

==== 🔍 예측 결과 ====
실제 라벨: 0  →  예측 라벨: 0
클래스 확률: 낙상(1): 0.0003, 비낙상(0): 0.9997
✅ 정상적인 행동입니다.

==== 🔍 예측 결과 ====
실제 라벨: 0  →  예측 라벨: 0
클래스 확률: 낙상(1): 0.0005, 비낙상(0): 0.9995
✅ 정상적인 행동입니다.

==== 🔍 예측 결과 ====
실제 라벨: 0  →  예측 라벨: 0
클래스 확률: 낙상(1): 0.0005, 비낙상(0): 0.9995
✅ 정상적인 행동입니다.

==== 🔍 예측 결과 ====
실제 라벨: 0  →  예측 라벨: 0
클래스 확률: 낙상(1): 0.0006, 비낙상(0): 0.9994
✅ 정상적인 행동입니다.

==== 🔍 예측 결과 ====
실제 라벨: 0  →  예측 라벨: 0
클래스 확

In [15]:
import cv2
import mediapipe as mp
VIDEO_PATH = BASE_DIR / "data" / "raw" / "LE2i" / "Home_01" / "Videos" / "video (1).avi"

In [29]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)
WINDOW_SIZE = 30

cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise RuntimeError(f"❌ 영상 파일을 열 수 없습니다: {VIDEO_PATH}")

keypoints_buffer = []

# ==============================
# 5️⃣ 프레임 루프
# ==============================
frameN = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    if results.pose_landmarks:
        # 33개 포인트 * (x,y,z)
        landmarks = results.pose_landmarks.landmark
        keypoints = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in landmarks]).flatten()
        keypoints_buffer.append(keypoints)

        if len(keypoints_buffer) >= WINDOW_SIZE:
            # 최근 WINDOW_SIZE 프레임 사용
            seq = np.stack(keypoints_buffer[-WINDOW_SIZE:], axis=0)  # (T, D)
            seq_scaled = scaler.transform(seq) if scaler else seq
            seq_scaled = np.expand_dims(seq_scaled, axis=0)  # (1, T, D)

            # 추론
            with torch.no_grad():
                x = torch.tensor(seq_scaled, dtype=torch.float32).to(device)
                logits = model(x)
                probs = torch.softmax(logits, dim=1)[0]
                pred = torch.argmax(probs).item()

            # 예측 결과 표시
            label_names = ["walk", "run", "jump", "sit", "stand"]  # 예시
            action = label_names[pred]
            cv2.putText(frame, f"Action: {action}", (20, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3, cv2.LINE_AA)
            if pred == 1:
                print(f"낙상 {frameN}")
            else:
                print("정상")

    #out.write(frame)
    cv2.imshow("Action Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frameN += 1
# ==============================
# 6️⃣ 종료 및 저장
# ==============================
cap.release()
#out.release()
cv2.destroyAllWindows()
print("✅ 영상 처리 완료! → output_result.mp4 저장됨")

C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


정상
정상


C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


정상
정상


C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


...

C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


낙상 258
낙상 259


C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


낙상 260
낙상 261
낙상 262
낙상 263
✅ 영상 처리 완료! → output_result.mp4 저장됨


C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
